In [66]:
import requests
import json
from bs4 import BeautifulSoup
from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest
from kinopoisk_unofficial.request.films.box_office_request import BoxOfficeRequest
import time
from selenium import webdriver
import pandas as pd
import re

In [93]:
browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')

url_api = "https://kinopoiskapiunofficial.tech/api/v2.2/films/"

links = []
films_raw_data = []

/var/folders/n0/z4q56_291yl18hx7bzl2b5b40000gn/T/ipykernel_87821/4123375354.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/kosarenok/Desktop/chromedriver_mac64/chromedriver')


In [108]:
list_of_links_n_budget = []

for year in range(2012, 2024):
    url = f"https://www.kinopoisk.ru/box/best_world/view_year/{year}/"
    browser.get(url)
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    counter = 0

    for a, b in zip(soup.findAll('a', class_='all'), soup.findAll('span', style="color: #f60; white-space: nowrap; display: block")):
        if counter < 25:
            link_n_budget = {
                "kinopoisk_id": a['href'].split('/')[2],
                "budget": int(re.sub(r'[$\xa0]',"",(b.get_text())))
            }
            list_of_links_n_budget.append(link_n_budget)
            counter += 1
        else:
            break


In [44]:
api_client = KinopoiskApiClient('505d7a7b-3062-43bf-81c2-04df36a9159e')

counter = 0

for i in range(0,len(links)):
    counter += 1
    if counter <= 10:
        request = FilmRequest(int(links[i]))
        response = api_client.films.send_film_request(request)
        films_raw_data.append(response.film)
    else:
        time.sleep(2)
        counter = 0

In [53]:
films_award_data = []

headers = {'X-API-KEY': '9eaa3824-1c1b-41ee-850e-e6b20e2ed3fc'}
counter = 0

for k in range(0,len(films_raw_data)):
    counter += 1
    if counter <= 10:
        request = requests.get(f'https://kinopoiskapiunofficial.tech/api/v2.2/films/{films_raw_data[k].kinopoisk_id}/awards', headers=headers)
        response = request.json()
        film_awards = {
                'kinopoisk_id' : films_raw_data[k].kinopoisk_id,
                'name' : 'Оскар'
            }
        for t in [x for x in range(len(response['items'])) if response['items'][x]['name'] == 'Оскар']:
            film_awards[response['items'][t]['nominationName']] = response['items'][t]['win']
            film_awards['year'] = response['items'][t]['year']
            films_award_data.append(film_awards)
    else:
        time.sleep(2)
        counter = 0

In [95]:
films_award_dataframe = pd.DataFrame(films_award_data)
films_award_dataframe = films_award_dataframe.drop_duplicates()
films_dataframe = pd.DataFrame(films_raw_data)
films_budget_dataframe = pd.DataFrame(list_of_links_n_budget)

In [96]:
films_budget_dataframe.rename(columns = {'link':'kinopoisk_id'}, inplace=True)
films_budget_dataframe = films_budget_dataframe.astype({'kinopoisk_id': int})

films_award_dataframe.rename(columns = {'kino_id':'kinopoisk_id'}, inplace=True)

In [102]:
films_with_budget = films_dataframe.merge(films_budget_dataframe, how='outer', on='kinopoisk_id')

In [103]:
all_films_data = films_with_budget.merge(films_award_dataframe, how='outer', on='kinopoisk_id')

In [104]:
# df.sort_values(by=['col1'])

all_films_data.sort_values(by=['year_x'], inplace=True)


In [105]:
all_films_data.to_excel("Kinopoisk_parsed_data_new.xlsx", sheet_name='raw_data')  